In [1]:
import osmium
import os
from itertools import islice

out_file = '../osm/divided.osm'
input_file = '../osm/export.osm'

In [2]:
node_usage = {}
class NodeIndexer(osmium.SimpleHandler):

    def way(self, w: osmium.Way):
        if 'highway' not in w.tags:
            return
        for i in w.nodes:
            node_usage.setdefault(i.ref, set()).add(w.id)

indexerHandler = NodeIndexer()
indexerHandler.apply_file(input_file)

In [3]:
class WayDivider(osmium.SimpleHandler):
    writer: osmium.SimpleWriter
    
    def __init__(self, writer: osmium.SimpleWriter):
        super().__init__()
        self.writer = writer
        self.new_id_ = 1
        self.exceptionCount = 0 
        self.ways_to_new: dict[int, set[int]] = {}
        self.node_usage_new: dict[int, set[int]] = {}

    def new_id(self):
        a = self.new_id_
        self.new_id_ += 1
        return a
        
    def node(self, n: osmium.Node):
        if n.id in node_usage:
            self.writer.add_node(n)

    def add_new_way(self, old_id, new_way: osmium.Way):
        self.writer.add_way(new_way)
        self.ways_to_new.setdefault(old_id, set()).add(new_way.id)
        for i in new_way.nodes:
            self.node_usage_new.setdefault(i.ref, set()).add(new_way.id)

    def way(self, w: osmium.Way):
        if 'highway' not in w.tags:
            return
        try:
            divided_to = 0
            divided = [w.nodes[0]]
            for i in islice(w.nodes, 1, None):
                divided.append(i)
                if len(node_usage[i.ref]) > 1:
                    self.add_new_way(w.id, w.replace(nodes=divided, id=self.new_id()))
                    divided = [i]
                    divided_to += 1
            if len(divided) > 1:
                self.add_new_way(w.id, w.replace(nodes=divided, id=self.new_id()))
                divided_to += 1
            if divided_to == 0:
                raise Exception(f"divided to 0: {w.id}")
            if divided_to < 2 and (w.nodes.ends_have_same_id() or w.is_closed()):
                #print(divided_to)
                raise Exception(f"w.nodes.ends_have_same_id() or w.is_closed(): {w.id}")
        except Exception as e:
            raise e
            self.exceptionCount = self.exeptionCount + 1 #когда добавил дома начали вылезать...

    def relation(self, r: osmium.Relation):
        if 'restriction' not in r.tags:
            return
        fr = None
        via = None
        to = None
        if len(r.members) > 3:
            print("bad relation (more than 3 members):", r.id)
            return
        for i in r.members:
            if i.role == "from":
                if not i.type == "w" or fr is not None:
                    print("bad relation:", r.id)
                fr = i.ref
            elif i.role == "to":
                if not i.type == "w" or to is not None:
                    print("bad relation:", r.id)
                to = i.ref
            elif i.role == "via":
                if not i.type == "n" or via is not None:
                    print("bad relation:", r.id)
                via = i.ref
        if via in self.node_usage_new and fr in self.ways_to_new and to in self.ways_to_new:
            new_fr = self.ways_to_new[fr] & self.node_usage_new[via]
            if len(new_fr) != 1:
                print("could not find exactly one from", r.id, (fr, via), (self.ways_to_new[fr], self.node_usage_new[via]))
                return
            new_fr = next(iter(new_fr))
            new_to = self.ways_to_new[to] & self.node_usage_new[via]
            if len(new_to) != 1:
                print("could not find exactly one to", r.id, (to, via), (self.ways_to_new[to], self.node_usage_new[via]))
                return
            new_to = next(iter(new_to))
            self.writer.add_relation(r.replace(members=[
                osmium.osm.RelationMember(new_fr, "w", "from"),
                osmium.osm.RelationMember(via, "n", "via"),
                osmium.osm.RelationMember(new_to, "w", "to"),
            ]))

In [4]:
try:
    os.remove(out_file)
except FileNotFoundError:
    pass

writer = osmium.SimpleWriter(out_file)
try:
    handler = WayDivider(writer)
    handler.apply_file(input_file, locations=True)
    print("Exceptions:", handler.exceptionCount)
finally:
    writer.close()

bad relation (more than 3 members): 1283498
bad relation (more than 3 members): 5685844
bad relation (more than 3 members): 5837923
could not find exactly one to 6583523 (442203049, 314953929) ({12467, 12468}, {7920, 15016, 9414, 1087})
bad relation: 9475817
Exceptions: 0
